# LLM Reflection

AutoGen supports different LLM prompting and reasoning strategies, such as ReAct, Reflection/Self-Critique, and more. This notebook demonstrates how to realize general LLM reflection with AutoGen. Reflection is a general prompting strategy which involves having LLMs analyze their own outputs, behaviors, knowledge, or reasoning processes.

This example leverages a generic interface [nested chats](/docs/tutorial/conversation-patterns#nested-chats) to implement the reflection using multi-agents.

In [ ]:
# %pip install "pyautogen>=0.2.18"

In [2]:
import os
import autogen
from autogen import AssistantAgent, UserProxyAgent
from autogen.coding import LocalCommandLineCodeExecutor, DockerCommandLineCodeExecutor

config_list = autogen.config_list_from_json(env_or_file="OAI_CONFIG_LIST")

llm_config = {
    "config_list": config_list,
    "timeout": 120,
}

## Construct Agents
Now we create three agents, including `user_proxy` as a user proxy, `writing_assistant` for generating solutions (based on the initial request or critique), and `reflection_assistant` for reflecting and providing critique.

In [3]:
os.makedirs("coding", exist_ok=True)
# Use DockerCommandLineCodeExecutor if docker is available to run the generated code.
# Using docker is safer than running the generated code directly.
# code_executor = DockerCommandLineCodeExecutor(work_dir="coding")
code_executor = LocalCommandLineCodeExecutor(work_dir="coding")

user_proxy = UserProxyAgent(
    name="user_proxy",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="TERMINATE",
    max_consecutive_auto_reply=10,
    code_execution_config={"executor": code_executor},
)

writing_assistant = AssistantAgent(
    name="writing_assistant",
    system_message="You are an writing assistant tasked to write engaging blogpost. You try generate the best blogpost possible for the user's request. If the user provides critique, respond with a revised version of your previous attempts.",
    llm_config=llm_config,
)

reflection_assistant = AssistantAgent(
    name="reflection_assistant",
    system_message="Generate critique and recommendations on the writing. Provide detailed recommendations, including requests for length, depth, style, etc..",
    llm_config=llm_config,
)

## Construct Agent Chats with `reflection_assistant` being a Nested Agent for Reflection

In [5]:
def reflection_message(recipient, messages, sender, config):
    print("Reflecting...")
    return f"Reflect and provide critique on the following writing. \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}"


nested_chat_queue = [
    {
        "recipient": reflection_assistant,
        "message": reflection_message,
        "max_turns": 1,
    },
]
user_proxy.register_nested_chats(
    nested_chat_queue,
    trigger=writing_assistant,
    # position=4,
)

from autogen.cache import Cache

# Use Cache.disk to cache the generated responses.
# This is useful when the same request to the LLM is made multiple times.
with Cache.disk(cache_seed=42) as cache:
    user_proxy.initiate_chat(
        writing_assistant,
        message="Write an engaging blogpost on the recent updates in AI. "
        "The blogpost should be engaging and understandable for general audience. "
        "Should have more than 3 paragraphes but no longer than 1000 words.",
        max_turns=2,
        cache=cache,
    )

user_proxy (to writing_assistant):

Write an engaging blogpost on the recent updates in AI. The blogpost should be engaging and understandable for general audience. Should have more than 3 paragraphes but no longer than 1000 words.

--------------------------------------------------------------------------------
writing_assistant (to user_proxy):

Title: "Unveiling the Evolution in AI: A Walkthrough of Exciting Advancements"

Hello adventurers of artificial intelligence! The field of AI has surfaced as a rapidly evolving realm. Wondering what’s new in the enigmatic yet exciting sphere of AI? Don’t worry!  I’m here to offer a captivating glimpse into the latest updates.

Our first pitstop in the AI journey is the groundbreaking advancement triggered by OpenAI. Earlier this year, they introduced the world to GPT-3, the most sophisticated language prediction model to date. Yes, a machine that can write like a human, imagine! Whether it's crafting an alien language for your next sci-fi nov